[View in Colaboratory](https://colab.research.google.com/github/nishanthjois/Deep-Learning-Advanced/blob/master/NLP_Sentiment_Analysis_IMDB.ipynb)

# TensorFlow Tutorial #20
# Natural Language Processing

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

## Introduction

https://www.youtube.com/watch?v=DDByc9LyMV8&list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ&index=27

This tutorial is about a basic form of Natural Language Processing (NLP) called Sentiment Analysis, in which we will try and classify a movie review as either positive or negative.

Consider a simple example: "This movie is not very good." This text ends with the words "very good" which indicates a very positive sentiment, but it is negated because it is preceded by the word "not", so the text should be classified as having a negative sentiment. How can we teach a Neural Network to do this classification?

Another problem is that neural networks cannot work directly on text-data, so we need to convert text into numbers that are compatible with a neural network.

Yet another problem is that a text may be arbitrarily long. The neural networks we have worked with in previous tutorials use fixed data-shapes - except for the first dimension of the data which varies with the batch-size. Now we need a type of neural network that can work on both short and long sequences of text.

You should be familiar with TensorFlow and Keras in general, see Tutorials #01 and #03-C.

## Flowchart

To solve this problem we need several processing steps. First we need to convert the raw text-words into so-called tokens which are integer values. These tokens are really just indices into a list of the entire vocabulary. Then we convert these integer-tokens into so-called embeddings which are real-valued vectors, whose mapping will be trained along with the neural network, so as to map words with similar meanings to similar embedding-vectors. Then we input these embedding-vectors to a Recurrent Neural Network which can take sequences of arbitrary length as input and output a kind of summary of what it has seen in the input. This output is then squashed using a Sigmoid-function to give us a value between 0.0 and 1.0, where 0.0 is taken to mean a negative sentiment and 1.0 means a positive sentiment. This whole process allows us to classify input-text as either having a negative or positive sentiment.

The flowchart of the algorithm is roughly:

**RAW TEXT > TOKENIZER > EMBEDDING > RNN > SIGMOID > OUTPUT

## Recurrent Neural Network

The basic building block in a Recurrent Neural Network (RNN) is a Recurrent Unit (RU). There are many different variants of recurrent units such as the rather clunky LSTM (Long-Short-Term-Memory) and the somewhat simpler GRU (Gated Recurrent Unit) which we will use in this tutorial. Experiments in the literature suggest that the LSTM and GRU have roughly similar performance. Even simpler variants also exist and the literature suggests that they may perform even better than both LSTM and GRU, but they are not implemented in Keras which we will use in this tutorial.

The following figure shows the abstract idea of a recurrent unit, which has an internal state that is being updated every time the unit receives a new input. This internal state serves as a kind of memory. However, it is not a traditional kind of computer memory which stores bits that are either on or off. Instead the recurrent unit stores floating-point values in its memory-state, which are read and written using matrix-operations so the operations are all differentiable. This means the memory-state can store arbitrary floating-point values (although typically limited between -1.0 and 1.0) and the network can be trained like a normal neural network using Gradient Descent.

The new state-value depends on both the old state-value and the current input. For example, if the state-value has memorized that we have recently seen the word "not" and the current input is "good" then we need to store a new state-value that memorizes "not good" which indicates a negative sentiment.

The part of the recurrent unit that is responsible for mapping old state-values and inputs to the new state-value is called a gate, but it is really just a type of matrix-operation. There is another gate for calculating the output-values of the recurrent unit. The implementation of these gates vary for different types of recurrent units. This figure merely shows the abstract idea of a recurrent unit. The LSTM has more gates than the GRU but some of them are apparently redundant so they can be omitted.

In order to train the recurrent unit, we must gradually change the weight-matrices of the gates so the recurrent unit gives the desired output for an input sequence. This is done automatically in TensorFlow.

![Recurrent unit](images/20_recurrent_unit.png)

### Unrolled Network

Another way to visualize and understand a Recurrent Neural Network is to "unroll" the recursion. In this figure there is only a single recurrent unit denoted RU, which will receive a text-word from the input sequence in a series of time-steps.

The initial memory-state of the RU is reset to zero internally by Keras / TensorFlow every time a new sequence begins.

In the first time-step the word "this" is input to the RU which uses its internal state (initialized to zero) and its gate to calculate the new state. The RU also uses its other gate to calculate the output but it is ignored here because it is only needed at the end of the sequence to output a kind of summary.

In the second time-step the word "is" is input to the RU which now uses the internal state that was just updated from seeing the previous word "this".

There is not much meaning in the words "this is" so the RU probably doesn't save anything important in its internal state from seeing these words. But when it sees the third word "not" the RU has learned that it may be important for determining the overall sentiment of the input-text, so it needs to be stored in the memory-state of the RU, which can be used later when the RU sees the word "good" in time-step 6.

Finally when the entire sequence has been processed, the RU outputs a vector of values that summarizes what it has seen in the input sequence. We then use a fully-connected layer with a Sigmoid activation to get a single value between 0.0 and 1.0 which we interpret as the sentiment either being negative (values close to 0.0) or positive (values close to 1.0).

Note that for the sake of clarity, this figure doesn't show the mapping from text-words to integer-tokens and embedding-vectors, as well as the fully-connected Sigmoid layer on the output.

![Unrolled network](images/20_unrolled_flowchart.png)

### 3-Layer Unrolled Network

In this tutorial we will use a Recurrent Neural Network with 3 recurrent units (or layers) denoted RU1, RU2 and RU3 in the "unrolled" figure below.

The first layer is much like the unrolled figure above for a single-layer RNN. First the recurrent unit RU1 has its internal state initialized to zero by Keras / TensorFlow. Then the word "this" is input to RU1 and it updates its internal state. Then it processes the next word "is", and so forth. But instead of outputting a single summary value at the end of the sequence, we use the output of RU1 for every time-step. This creates a new sequence that can then be used as input for the next recurrent unit RU2. The same process is repeated for the second layer and this creates a new output sequence which is then input to the third layer's recurrent unit RU3, whose final output is passed to a fully-connected Sigmoid layer that outputs a value between 0.0 (negative sentiment) and 1.0 (positive sentiment).

Note that for the sake of clarity, the mapping of text-words to integer-tokens and embedding-vectors has been omitted from this figure.

![Unrolled 3-layer network](images/20_unrolled_3layers_flowchart.png)

### Exploding & Vanishing Gradients

In order to train the weights for the gates inside the recurrent unit, we need to minimize some loss-function which measures the difference between the actual output of the network relative to the desired output.

From the "unrolled" figures above we see that the reccurent units are applied recursively for each word in the input sequence. This means the recurrent gate is applied once for each time-step. The gradient-signals have to flow back from the loss-function all the way to the first time the recurrent gate is used. If the gradient of the recurrent gate is multiplicative, then we essentially have an exponential function.

In this tutorial we will use texts that have more than 500 words. This means the RU's gate for updating its internal memory-state is applied recursively more than 500 times. If a gradient of just 1.01 is multiplied with itself 500 times then it gives a value of about 145. If a gradient of just 0.99 is multiplied with itself 500 times then it gives a value of about 0.007. These are called exploding and vanishing gradients. The only gradients that can survive recurrent multiplication are 0 and 1.

To avoid these so-called exploding and vanishing gradients, care must be made when designing the recurrent unit and its gates. That is why the actual implementation of the GRU is more complicated, because it tries to send the gradient back through the gates without this distortion.

## Imports

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

We need to import several things from Keras.

In [0]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

This was developed using Python 3.6 (Anaconda) and package versions:

In [9]:
tf.__version__

'1.6.0'

In [10]:
tf.keras.__version__

'2.1.3-tf'

In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpkqaieez8/pubring.gpg' created
gpg: /tmp/tmpkqaieez8/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive



fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [8]:
!ls drive/data

20_Natural_Language_Processing (1).ipynb  __init__.py
cache.py				  knifey.py
cifar10.py				  movie_titles.csv
coco.py					  my_nlp_model.h5
convert.py				  __pycache__
dataset					  reinforcement_learning.py
dataset.py				  test1.csv
data_small.csv				  test_sparse_matrix.npz
download.py				  train1.csv
europarl.py				  train_sparse_matrix.npz
imdb.py					  vgg16.py
inception5h.py				  weather.py
inception.py


## Load Data

We will use a data-set consisting of 50000 reviews of movies from IMDB. Keras has a built-in function for downloading a similar data-set (but apparently half the size). However, Keras' version has already converted the text in the data-set to integer-tokens, which is a crucial part of working with natural languages that will also be demonstrated in this tutorial, so we download the actual text-data.

NOTE: The data-set is 84 MB and will be downloaded automatically.

Change this if you want the files saved in another directory.

In [0]:
# imdb.data_dir = "data/IMDB/"

In [9]:
!ls


data  datalab  drive


Automatically download and extract the files.

In [0]:
########################################################################
#
# Functions for downloading and extracting data-files from the internet.
#
# Implemented in Python 3.5
#
########################################################################
#
# This file is part of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2016 by Magnus Erik Hvass Pedersen
#
########################################################################

import sys
import os
import urllib.request
import tarfile
import zipfile

########################################################################


def _print_download_progress(count, block_size, total_size):
    """
    Function used for printing the download progress.
    Used as a call-back function in maybe_download_and_extract().
    """

    # Percentage completion.
    pct_complete = float(count * block_size) / total_size

    # Status-message. Note the \r which means the line should overwrite itself.
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()


########################################################################


def maybe_download_and_extract(url, download_dir):
    """
    Download and extract the data if it doesn't already exist.
    Assumes the url is a tar-ball file.

    :param url:
        Internet URL for the tar-file to download.
        Example: "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

    :param download_dir:
        Directory where the downloaded file is saved.
        Example: "data/CIFAR-10/"

    :return:
        Nothing.
    """

    # Filename for saving the file downloaded from the internet.
    # Use the filename from the URL and add it to the download_dir.
    filename = url.split('/')[-1]
    file_path = os.path.join(download_dir, filename)

    # Check if the file already exists.
    # If it exists then we assume it has also been extracted,
    # otherwise we need to download and extract it now.
    if not os.path.exists(file_path):
        # Check if the download directory exists, otherwise create it.
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        # Download the file from the internet.
        file_path, _ = urllib.request.urlretrieve(url=url,
                                                  filename=file_path,
                                                  reporthook=_print_download_progress)

        print()
        print("Download finished. Extracting files.")

        if file_path.endswith(".zip"):
            # Unpack the zip-file.
            zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
        elif file_path.endswith((".tar.gz", ".tgz")):
            # Unpack the tar-ball.
            tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)

        print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")


########################################################################


########################################################################
#
# Functions for downloading the IMDB Review data-set from the internet
# and loading it into memory.
#
# Implemented in Python 3.6
#
# Usage:
# 1) Set the variable data_dir with the desired storage directory.
# 2) Call maybe_download_and_extract() to download the data-set
#    if it is not already located in the given data_dir.
# 3) Call load_data(train=True) to load the training-set.
# 4) Call load_data(train=False) to load the test-set.
# 5) Use the returned data in your own program.
#
# Format:
# The IMDB Review data-set consists of 50000 reviews of movies
# that are split into 25000 reviews for the training- and test-set,
# and each of those is split into 12500 positive and 12500 negative reviews.
# These are returned as lists of strings by the load_data() function.
#
########################################################################
#
# This file is part of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2018 by Magnus Erik Hvass Pedersen
#
########################################################################

import os
import glob

########################################################################

# Directory where you want to download and save the data-set.
# Set this before you start calling any of the functions below.
data_dir = "data/IMDB/"

# URL for the data-set on the internet.
data_url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"


########################################################################
# Private helper-functions.

def _read_text_file(path):
    """
    Read and return all the contents of the text-file with the given path.
    It is returned as a single string where all lines are concatenated.
    """

    with open(path, 'rt') as file:
        # Read a list of strings.
        lines = file.readlines()

        # Concatenate to a single string.
        text = " ".join(lines)

    return text


########################################################################
# Public functions that you may call to download the data-set from
# the internet and load the data into memory.


def maybe_download_and_extract1():
    """
    Download and extract the IMDB Review data-set if it doesn't already exist
    in data_dir (set this variable first to the desired directory).
    """

    maybe_download_and_extract(url=data_url, download_dir=data_dir)


def load_data(train=True):
    """
    Load all the data from the IMDB Review data-set for sentiment analysis.

    :param train: Boolean whether to load the training-set (True)
                  or the test-set (False).

    :return:      A list of all the reviews as text-strings,
                  and a list of the corresponding sentiments
                  where 1.0 is positive and 0.0 is negative.
    """

    # Part of the path-name for either training or test-set.
    train_test_path = "train" if train else "test"

    # Base-directory where the extracted data is located.
    dir_base = os.path.join(data_dir, "aclImdb", train_test_path)

    # Filename-patterns for the data-files.
    path_pattern_pos = os.path.join(dir_base, "pos", "*.txt")
    path_pattern_neg = os.path.join(dir_base, "neg", "*.txt")

    # Get lists of all the file-paths for the data.
    paths_pos = glob.glob(path_pattern_pos)
    paths_neg = glob.glob(path_pattern_neg)

    # Read all the text-files.
    data_pos = [_read_text_file(path) for path in paths_pos]
    data_neg = [_read_text_file(path) for path in paths_neg]

    # Concatenate the positive and negative data.
    x = data_pos + data_neg

    # Create a list of the sentiments for the text-data.
    # 1.0 is a positive sentiment, 0.0 is a negative sentiment.
    y = [1.0] * len(data_pos) + [0.0] * len(data_neg)

    return x, y


########################################################################


In [31]:
maybe_download_and_extract1()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [30]:
!ls data/

europarl


Load the training- and test-sets.

In [0]:
x_train_text, y_train = load_data(train=True)
x_test_text, y_test = load_data(train=False)

In [34]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


Combine into one data-set for some uses below.

In [0]:
data_text = x_train_text + x_test_text

Print an example from the training-set to see that the data looks correct.

In [36]:
x_train_text[1]

"It ran 8 seasons, but it's first, in early 1959, and it's last, in the autumn of 1965, were shorter than seasons 2-7. CBS chief William Paley canceled Rawhide's production after watching the 1st show of season 8, in September, 1965, because he disliked the series without Eric Fleming as Gil Favor, who had departed after season 7. The last new episode aired in November, 1965. The lone 1966 CBS broadcast, on January 4, 1966, was a rerun. <br /><br />I have often wondered why Rawhide didn't switch to color filming for it's last season? Most of the big westerns of the 1960s had gone over to color by 1965. CBS was broadcasting in color that autumn, for many of their sitcoms, but westerns like Gunsmoke and Rawhide remained in black and white. Gunsmoke was the last western (and last prime time network series to switch to color) on September 17, 1966, for the episode Snap Decision."

The true "class" is a sentiment of the movie-review. It is a value of 0.0 for a negative sentiment and 1.0 for a positive sentiment. In this case the review is positive.

In [37]:
y_train[1]

1.0

## Tokenizer

A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set.

In [0]:
num_words = 10000

In [0]:
tokenizer = Tokenizer(num_words=num_words)

The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.

Note that we fit the tokenizer on the entire data-set so it gathers words from both the training- and test-data. This is OK as we are merely building a vocabulary and want it to be as complete as possible. The actual neural network will of course only be trained on the training-set.

In [40]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 9.92 s, sys: 103 ms, total: 10 s
Wall time: 10 s


If you want to use the entire vocabulary then set `num_words=None` above, and then it will automatically be set to the vocabulary-size here. (This is because of Keras' somewhat awkward implementation.)

In [0]:
if num_words is None:
    num_words = len(tokenizer.word_index)

We can then inspect the vocabulary that has been gathered by the tokenizer. This is ordered by the number of occurrences of the words in the data-set. These integer-numbers are called word indices or "tokens" because they uniquely identify each word in the vocabulary.

In [42]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

We can then use the tokenizer to convert all texts in the training-set to lists of these tokens.

In [0]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

For example, here is a text from the training-set:

In [44]:
x_train_text[1]

"It ran 8 seasons, but it's first, in early 1959, and it's last, in the autumn of 1965, were shorter than seasons 2-7. CBS chief William Paley canceled Rawhide's production after watching the 1st show of season 8, in September, 1965, because he disliked the series without Eric Fleming as Gil Favor, who had departed after season 7. The last new episode aired in November, 1965. The lone 1966 CBS broadcast, on January 4, 1966, was a rerun. <br /><br />I have often wondered why Rawhide didn't switch to color filming for it's last season? Most of the big westerns of the 1960s had gone over to color by 1965. CBS was broadcasting in color that autumn, for many of their sitcoms, but westerns like Gunsmoke and Rawhide remained in black and white. Gunsmoke was the last western (and last prime time network series to switch to color) on September 17, 1966, for the episode Snap Decision."

This text corresponds to the following list of tokens:

In [45]:
np.array(x_train_tokens[1])

array([   9, 2119,  723, 2499,   18,   44,   86,    8,  404, 6056,    2,
         44,  238,    8,    1,    4, 8602,   70, 5567,   71, 2499,  230,
        702, 6639, 2305, 1012, 5944,  353,  100,  147,    1, 3407,  119,
          4,  882,  723,    8, 5541, 8602,   84,   28, 4989,    1,  204,
        208, 2150,   14, 2007,   35,   66,  100,  882,  702,    1,  238,
        168,  398, 3110,    8, 9408, 8602,    1, 3368, 7952, 6639, 3551,
         20, 9015,  454, 7952,   13,    3,    7,    7,   10,   25,  403,
       3732,  134,  154, 4379,    5, 1455, 1385,   15,   44,  238,  882,
         88,    4,    1,  191, 2512,    4,    1, 4557,   66,  781,  121,
          5, 1455,   31, 8602, 6639,   13,    8, 1455,   12,   15,  106,
          4,   65, 6126,   18, 2512,   37,    2, 4636,    8,  317,    2,
        452,   13,    1,  238,  895,    2,  238, 2443,   55, 2274,  204,
          5, 4379,    5, 1455,   20, 5541, 2962, 7952,   15,    1,  398,
       8770, 2105])

We also need to convert the texts in the test-set to tokens.

In [46]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)
print(x_test_text[1])
np.array(x_test_tokens[5])

'War movie' is a Hollywood genre that has been done and redone so many times that clichéd dialogue, rehashed plot and over-the-top action sequences seem unavoidable for any conflict dealing with large-scale combat. Once in a while, however, a war movie comes along that goes against the grain and brings a truly original and compelling story to life on the silver screen. The Civil War-era "Cold Mountain," starring Jude Law, Nicole Kidman and Renée Zellweger is such a film.<br /><br />Then again, calling Cold Mountain" a war movie is not entirely accurate. True enough, the film opens with a (quite literally) quick-and-dirty battle sequence that puts "Glory" director Edward Zwick shame. However, "Cold Mountain" is not so much about the Civil War itself as it is about the period and the people of the times. The story centers around disgruntled Confederate soldier Inman, played by Jude Law, who becomes disgusted with the gruesome war and homesick for the beautiful hamlet of Cold Mountain, No

array([  11,    6,    3,   78,   17,   15,   29, 6546,  455,   11,   17,
         45,    3,   78,  174,    4,  186,  150,    3,  547,    4, 2290,
       1168,    5, 2257,    3, 2144,    5,   76,   47, 1552, 2065,   18,
         29,  269,  356,   50,    3, 2539,  217,  319,    2,   33,  187])

## Padding and Truncating Data

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [47]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
print (num_tokens)

[177 145 111 ... 221 248 212]


The average number of tokens in a sequence is:

In [48]:
np.mean(num_tokens)

221.27716

The maximum number of tokens in a sequence is:

In [49]:
np.max(num_tokens)

2208

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [50]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

544

This covers about 95% of the data-set.

In [51]:
np.sum(num_tokens<max_tokens)/len(num_tokens)

0.94534

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [0]:
pad = 'pre'

In [0]:
x_train_pad=pad_sequences(x_train_tokens,maxlen=max_tokens,padding=pad,truncating=pad)

In [0]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [55]:
#x_train_tokens1 = []
x_train_tokens1 = np.asarray([[1, 2, 3]])
x_train_pad1=pad_sequences(x_train_tokens1,maxlen=20,padding=pad,truncating='pre')
x_train_pad1


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3]],
      dtype=int32)

We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [56]:
x_train_pad.shape

(25000, 544)

The matrix for the test-set has the same shape:

In [57]:
x_test_pad.shape

(25000, 544)

For example, we had the following sequence of tokens above:

In [58]:
np.array(x_train_tokens[67])

array([8097, 1453,   82, 1455, 1680,    4,   24,  667, 1402,    4,  876,
          2, 6473,    4,  614,    2,   15,    3, 2326,  187,   20, 8097,
       7423,   15,    3,  396,   12,   13,    8,   56,  664,   24,   12,
          6,    5,  131,   10,  229,   14,   43, 8097,  461,    5,   94,
         32,  435,  867, 1780,  867, 3794,   11,   17,    6,    3, 1347,
       5600,    8,   11, 7837,  650,    4,  320,  158,  136,  136,  109,
       4993,    2, 2326,   13,  130,  218,    2,   69, 1780,    2,  136,
       9947,  826, 2386,    1,    2, 1099, 3577,    4, 2270,  208,  109,
         96, 1270, 2705,   38,  348,   77,   45,   29,    1,   12,   27,
         98,  525,   16,   92, 1468,   16,  619,    4,    1, 1728, 2376,
          4,  952, 8097, 1321,    7,    7,   26, 2730,   11,    6,   21,
          3,   17,   12, 8097,  142,   26, 6424,   31])

This has simply been padded to create the following sequence. Note that when this is input to the Recurrent Neural Network, then it first inputs a lot of zeros. If we had padded 'post' then it would input the integer-tokens first and then a lot of zeros. This may confuse the Recurrent Neural Network.

In [59]:
x_train_pad[67]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## Tokenizer Inverse Map

For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [60]:
idx=tokenizer.word_index
print(idx)
inverse_map = dict(zip(idx.values(),idx.keys()))


{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'br': 7, 'in': 8, 'it': 9, 'i': 10, 'this': 11, 'that': 12, 'was': 13, 'as': 14, 'for': 15, 'with': 16, 'movie': 17, 'but': 18, 'film': 19, 'on': 20, 'not': 21, 'you': 22, 'are': 23, 'his': 24, 'have': 25, 'be': 26, 'one': 27, 'he': 28, 'all': 29, 'at': 30, 'by': 31, 'an': 32, 'they': 33, 'so': 34, 'who': 35, 'from': 36, 'like': 37, 'or': 38, 'just': 39, 'her': 40, 'out': 41, 'about': 42, 'if': 43, "it's": 44, 'has': 45, 'there': 46, 'some': 47, 'what': 48, 'good': 49, 'when': 50, 'more': 51, 'very': 52, 'up': 53, 'no': 54, 'time': 55, 'my': 56, 'even': 57, 'would': 58, 'she': 59, 'which': 60, 'only': 61, 'really': 62, 'see': 63, 'story': 64, 'their': 65, 'had': 66, 'can': 67, 'me': 68, 'well': 69, 'were': 70, 'than': 71, 'much': 72, 'we': 73, 'bad': 74, 'been': 75, 'get': 76, 'do': 77, 'great': 78, 'other': 79, 'will': 80, 'also': 81, 'into': 82, 'people': 83, 'because': 84, 'how': 85, 'first': 86, 'him': 87, 'most': 88, "don't":

In [61]:
inverse_map[11]

'this'

Helper-function for converting a list of tokens back to a string of words.

In [0]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if tokens!=0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

For example, this is the original text from the data-set:

In [63]:
x_train_text[1]

"It ran 8 seasons, but it's first, in early 1959, and it's last, in the autumn of 1965, were shorter than seasons 2-7. CBS chief William Paley canceled Rawhide's production after watching the 1st show of season 8, in September, 1965, because he disliked the series without Eric Fleming as Gil Favor, who had departed after season 7. The last new episode aired in November, 1965. The lone 1966 CBS broadcast, on January 4, 1966, was a rerun. <br /><br />I have often wondered why Rawhide didn't switch to color filming for it's last season? Most of the big westerns of the 1960s had gone over to color by 1965. CBS was broadcasting in color that autumn, for many of their sitcoms, but westerns like Gunsmoke and Rawhide remained in black and white. Gunsmoke was the last western (and last prime time network series to switch to color) on September 17, 1966, for the episode Snap Decision."

We can recreate this text except for punctuation and other symbols, by converting the list of tokens back to words:

In [64]:
tokens_to_string(x_train_tokens[1])

"it ran 8 seasons but it's first in early 1959 and it's last in the of 1965 were shorter than seasons 2 7 cbs chief william canceled production after watching the 1st show of season 8 in september 1965 because he disliked the series without eric as favor who had after season 7 the last new episode aired in november 1965 the lone 1966 cbs broadcast on january 4 1966 was a br br i have often wondered why didn't switch to color filming for it's last season most of the big westerns of the 1960s had gone over to color by 1965 cbs was in color that for many of their sitcoms but westerns like and remained in black and white was the last western and last prime time network series to switch to color on september 17 1966 for the episode snap decision"

## Create the Recurrent Neural Network

We are now ready to create the Recurrent Neural Network (RNN). We will use the Keras API for this because of its simplicity. See Tutorial #03-C for a tutorial on Keras.

In [0]:
model = Sequential()

The first layer in the RNN is a so-called Embedding-layer which converts each integer-token into a vector of values. This is necessary because the integer-tokens may take on values between 0 and 10000 for a vocabulary of 10000 words. The RNN cannot work on values in such a wide range. The embedding-layer is trained as a part of the RNN and will learn to map words with similar semantic meanings to similar embedding-vectors, as will be shown further below.

First we define the size of the embedding-vector for each integer-token. In this case we have set it to 8, so that each integer-token will be converted to a vector of length 8. The values of the embedding-vector will generally fall roughly between -1.0 and 1.0, although they may exceed these values somewhat.

The size of the embedding-vector is typically selected between 100-300, but it seems to work reasonably well with small values for Sentiment Analysis.

In [0]:
embedding_size = 8

The embedding-layer also needs to know the number of words in the vocabulary (`num_words`) and the length of the padded token-sequences (`max_tokens`). We also give this layer a name because we need to retrieve its weights further below.

In [73]:
print (num_words, max_tokens)
model.add(Embedding(input_dim=num_words,output_dim=embedding_size,input_length=max_tokens))

10000 544


We can now add the first Gated Recurrent Unit (GRU) to the network. This will have 16 outputs. Because we will add a second GRU after this one, we need to return sequences of data because the next GRU expects sequences as its input.

In [0]:
model.add(GRU(units=16,return_sequences=True))

This adds the second GRU with 8 output units. This will be followed by another GRU so it must also return sequences.

In [0]:
model.add(GRU(units=8, return_sequences=True))

This adds the third and final GRU with 4 output units. This will be followed by a dense-layer, so it should only give the final output of the GRU and not a whole sequence of outputs.

In [0]:
model.add(GRU(units=4))

Add a fully-connected / dense layer which computes a value between 0.0 and 1.0 that will be used as the classification output.

In [0]:
model.add(Dense(1,activation='sigmoid'))

Use the Adam optimizer with the given learning-rate.

In [0]:
optimizer=Adam(lr=0.001)

Compile the Keras model so it is ready for training.

In [0]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [80]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 544, 8)            80000     
_________________________________________________________________
gru_5 (GRU)                  (None, None, 16)          1200      
_________________________________________________________________
gru_6 (GRU)                  (None, None, 8)           600       
_________________________________________________________________
gru_7 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


## Train the Recurrent Neural Network

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set.

In [81]:
%%time
model.fit(x=x_train_pad,y=y_train,validation_split=0.05,epochs=4,batch_size=64)

Train on 23750 samples, validate on 1250 samples
Epoch 1/4
23750/23750 [==============================]23750/23750 [==============================] - 1110s 47ms/step - loss: 0.6614 - acc: 0.5862 - val_loss: 0.5254 - val_acc: 0.7152

Epoch 2/4
 5056/23750 [=====>........................] 5056/23750 [=====>........................] - ETA: 14:35 - loss: 0.5141 - acc: 0.7561

23750/23750 [==============================]23750/23750 [==============================] - 1140s 48ms/step - loss: 0.4529 - acc: 0.7976 - val_loss: 0.3156 - val_acc: 0.8776

Epoch 3/4
12544/23750 [==============>...............]12544/23750 [==============>...............] - ETA: 9:09 - loss: 0.3414 - acc: 0.8620

23750/23750 [==============================]23750/23750 [==============================] - 1175s 49ms/step - loss: 0.3247 - acc: 0.8696 - val_loss: 0.2707 - val_acc: 0.9032

Epoch 4/4
15680/23750 [==================>...........]15680/23750 [==================>...........] - ETA: 6:26 - loss: 0.2609 - acc: 0.9017

23750/23750 [==============================]23750/23750 [==============================] - 1153s 49ms/step - loss: 0.2594 - acc: 0.9024 - val_loss: 0.2203 - val_acc: 0.9144

CPU times: user 1h 28min 43s, sys: 16min 7s, total: 1h 44min 51s
Wall time: 1h 16min 20s


## Performance on Test-Set

Now that the model has been trained we can calculate its classification accuracy on the test-set.

In [82]:
%%time
result = model.evaluate(x=x_test_pad,y=y_test)

25000/25000 [==============================]25000/25000 [==============================] - 578s 23ms/step

CPU times: user 8min 7s, sys: 1min 39s, total: 9min 46s
Wall time: 9min 37s


In [0]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 86.71%


In [83]:
from keras.models import load_model

model.save('drive/data/my_nlp_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model



Using TensorFlow backend.


In [0]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('drive/data/my_nlp_model.h5')

## Example of Mis-Classified Text

In order to show an example of mis-classified text, we first calculate the predicted sentiment for the first 1000 texts in the test-set.

In [79]:
%%time


y_pred = model.predict(x=x_test_pad[0:1000])


CPU times: user 19.3 s, sys: 4.14 s, total: 23.5 s
Wall time: 23.1 s


In [80]:
print (y_pred [:10])

[[0.65867335]
 [0.8131124 ]
 [0.31202817]
 [0.2482007 ]
 [0.2953085 ]
 [0.70128095]
 [0.9252687 ]
 [0.96709275]
 [0.9434258 ]
 [0.77194184]]


In [81]:
y_pred = y_pred.T[0]
print (y_pred)

[0.65867335 0.8131124  0.31202817 0.2482007  0.2953085  0.70128095
 0.9252687  0.96709275 0.9434258  0.77194184 0.8424696  0.5028162
 0.7886405  0.9166498  0.60726947 0.968182   0.9676518  0.965896
 0.8237442  0.64030457 0.9525035  0.10634927 0.9732915  0.1007164
 0.9324031  0.830928   0.9324845  0.9685722  0.92923224 0.9566816
 0.8697377  0.8237552  0.9653656  0.96915305 0.9724549  0.94793385
 0.96815467 0.957315   0.8523488  0.600168   0.31189194 0.9705467
 0.96966434 0.9628249  0.48519808 0.97237027 0.16473502 0.9418116
 0.9657955  0.07908642 0.8849709  0.16896154 0.23632547 0.9102609
 0.96888006 0.8877369  0.97077507 0.9565568  0.93528694 0.85211045
 0.8101422  0.8959148  0.9426853  0.75354266 0.74548626 0.9659365
 0.9315193  0.6896267  0.9694762  0.749541   0.96874833 0.5035556
 0.9723426  0.8909041  0.45294684 0.9346678  0.8784295  0.96818393
 0.9412824  0.96545905 0.9698693  0.9715759  0.34510088 0.6058897
 0.6034122  0.8637186  0.8822455  0.8605492  0.887203   0.63192487
 0.951

These predicted numbers fall between 0.0 and 1.0. We use a cutoff / threshold and say that all values above 0.5 are taken to be 1.0 and all values below 0.5 are taken to be 0.0. This gives us a predicted "class" of either 0.0 or 1.0.

In [0]:
cls_pred=np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])


The true "class" for the first 1000 texts in the test-set are needed for comparison.

In [0]:
cls_true = np.array(y_test[0:1000])

We can then get indices for all the texts that were incorrectly classified by comparing all the "classes" of these two arrays.

In [88]:
incorrect = np.where(cls_pred!=cls_true)
print(incorrect)
incorrect=incorrect[0]

(array([  2,   3,   4,  21,  23,  40,  44,  46,  49,  51,  52,  74,  82,
        91,  95,  96, 104, 108, 112, 127, 131, 132, 140, 150, 157, 158,
       163, 170, 171, 176, 179, 180, 181, 182, 185, 187, 188, 191, 197,
       200, 204, 210, 212, 217, 220, 225, 228, 229, 246, 250, 251, 255,
       256, 259, 261, 262, 267, 284, 286, 297, 305, 306, 309, 311, 336,
       341, 344, 347, 351, 352, 356, 358, 359, 364, 366, 368, 371, 373,
       382, 384, 388, 390, 394, 404, 416, 417, 424, 425, 430, 433, 435,
       437, 441, 455, 456, 466, 467, 469, 478, 479, 480, 481, 485, 486,
       487, 488, 496, 497, 499, 500, 511, 514, 523, 528, 531, 539, 542,
       545, 546, 551, 557, 564, 574, 576, 580, 581, 582, 587, 592, 593,
       596, 609, 617, 620, 624, 625, 630, 632, 638, 644, 649, 657, 658,
       659, 665, 676, 677, 683, 685, 687, 689, 692, 693, 709, 711, 715,
       718, 720, 727, 735, 738, 739, 741, 742, 744, 750, 751, 752, 759,
       763, 773, 777, 780, 782, 790, 791, 796, 797, 798, 800, 8

Of the 1000 texts used, how many were mis-classified?

In [89]:
len(incorrect)

226

Let us look at the first mis-classified text. We will use its index several times.

In [90]:
idx = incorrect[0]
idx

2

The mis-classified text is:

In [91]:
text = x_test_text[idx]
text

"If you are going to watch this film because Michael Caine or Michael Gambon are in it then don't bother, it's not their typical role although I found Gambon fantastic. Instead watch it for Dylan Moran I am a fan of everything I have seen him in and this is no exception, I didn't even realise he could act but even the characters which he has to pass himself off as I found completely believable, which is impressive considering the audience knows they are fake.<br /><br />The plot is genius and although it is not constant laughs all the way through it has plenty of other charms. A great film for people with a sense of humour."

These are the predicted and true classes for the text:

In [92]:
y_pred[idx]

0.31202817

In [93]:
cls_true[idx]

1.0

## New Data

Let us try and classify new texts that we make up. Some of these are obvious, while others use negation and sarcasm to try and confuse the model into mis-classifying the text.

In [0]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

We first convert these texts to arrays of integer-tokens because that is needed by the model.

In [96]:
tokens = tokenizer.texts_to_sequences(texts)
print (tokens)

[[11, 17, 6, 799, 10, 62, 37, 9, 84, 9, 6, 34, 49], [49, 17], [273, 10, 37, 11, 17], [], [43, 10, 70, 3, 1940, 2188, 91, 11, 17, 233, 26, 49], [74, 17], [21, 3, 49, 17], [11, 17, 62, 1693, 67, 10, 76, 56, 290, 141, 591]]


To input texts with different lengths into the model, we also need to pad and truncate them.

In [97]:
print(max_tokens,pad)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

544 pre


(8, 544)

In [98]:
print(tokens_pad)

[[  0   0   0 ...   6  34  49]
 [  0   0   0 ...   0  49  17]
 [  0   0   0 ...  37  11  17]
 ...
 [  0   0   0 ...   0  74  17]
 [  0   0   0 ...   3  49  17]
 [  0   0   0 ... 290 141 591]]


We can now use the trained model to predict the sentiment for these texts.

In [99]:
model.predict(tokens_pad)

array([[0.86458445],
       [0.66036963],
       [0.29226708],
       [0.5732741 ],
       [0.4146285 ],
       [0.19331142],
       [0.5921217 ],
       [0.12962118]], dtype=float32)

A value close to 0.0 means a negative sentiment and a value close to 1.0 means a positive sentiment. These numbers will vary every time you train the model.

## Embeddings

The model cannot work on integer-tokens directly, because they are integer values that may range between 0 and the number of words in our vocabulary, e.g. 10000. So we need to convert the integer-tokens into vectors of values that are roughly between -1.0 and 1.0 which can be used as input to a neural network.

This mapping from integer-tokens to real-valued vectors is also called an "embedding". It is essentially just a matrix where each row contains the vector-mapping of a single token. This means we can quickly lookup the mapping of each integer-token by simply using the token as an index into the matrix. The embeddings are learned along with the rest of the model during training.

Ideally the embedding would learn a mapping where words that are similar in meaning also have similar embedding-values. Let us investigate if that has happened here.

First we need to get the embedding-layer from the model:

In [0]:
#layer_embedding = model.get_layer('layer_embedding')
layer_embedding = model.get_layer('embedding_2')

We can then get the weights used for the mapping done by the embedding-layer.

In [120]:
weights_embedding = layer_embedding.get_weights()[0]
print(weights_embedding)

[[ 0.26473242  0.49662563  0.20638427 ...  0.81207025 -0.08719666
   0.19814788]
 [ 0.01793321  0.29731637  0.62950253 ...  0.97221947 -0.01220716
   0.1148654 ]
 [ 1.0045968   0.71746904  0.8751119  ...  0.69342566  0.17486723
   0.4128322 ]
 ...
 [ 0.41636074  0.56854826  0.4850428  ...  0.81995535  0.90533906
   0.68799585]
 [ 0.7963472   0.1253027   0.74226505 ...  0.04114349  0.8067641
   0.06165722]
 [ 0.67313194  0.9171713   0.66024864 ...  0.6768927   0.6247592
   0.34302297]]


Note that the weights are actually just a matrix with the number of words in the vocabulary times the vector length for each embedding. That's because it is basically just a lookup-matrix.

In [109]:
weights_embedding.shape

(10000, 8)

Let us get the integer-token for the word 'good', which is just an index into the vocabulary.

In [116]:
token_good = tokenizer.word_index['awesome']
token_good

1171

Let us also get the integer-token for the word 'great'.

In [111]:
token_great = tokenizer.word_index['great']
token_great

78

These integertokens may be far apart and will depend on the frequency of those words in the data-set.

Now let us compare the vector-embeddings for the words 'good' and 'great'. Several of these values are similar, although some values are quite different. Note that these values will change every time you train the model.

In [121]:
weights_embedding[token_good]

array([0.9680488 , 0.35955897, 0.5620765 , 0.4630551 , 0.96129405,
       0.7485588 , 0.946915  , 0.32198286], dtype=float32)

In [122]:
weights_embedding[token_great]

array([ 1.0631231 ,  0.47565678,  0.7126732 ,  1.2699968 ,  1.426379  ,
       -0.21672449,  0.8865152 , -0.2785172 ], dtype=float32)

Similarly, we can compare the embeddings for the words 'bad' and 'horrible'.

In [0]:
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [124]:
weights_embedding[token_bad]

array([ 0.5223386 ,  0.76654005, -0.08809828,  0.33518857, -0.07621623,
        1.142144  ,  0.05902198,  0.50867933], dtype=float32)

In [125]:
weights_embedding[token_horrible]

array([ 0.391157  ,  0.5864746 , -0.00261769,  0.14589658,  0.2627825 ,
        0.50006855, -0.20981513,  1.4311601 ], dtype=float32)

### Sorted Words

We can also sort all the words in the vocabulary according to their "similarity" in the embedding-space. We want to see if words that have similar embedding-vectors also have similar meanings.

Similarity of embedding-vectors can be measured by different metrics, e.g. Euclidean distance or cosine distance.

We have a helper-function for calculating these distances and printing the words in sorted order.

In [0]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

We can then print the words that are near and far from the word 'great' in terms of their vector-embeddings. Note that these may change each time you train the model.

In [127]:
print_sorted_words('great', metric='cosine')

Distance from 'great':
-0.000 - great
0.024 - screened
0.030 - 7
0.032 - amusement
0.035 - delivers
0.037 - growth
0.040 - shiny
0.040 - bynes
0.041 - regret
0.043 - mcdowell
...
1.047 - ha
1.061 - running
1.062 - awful
1.076 - compassion
1.096 - winning
1.138 - terrible
1.166 - worst
1.189 - failed
1.202 - waste
1.312 - dull


Similarly, we can print the words that are near and far from the word 'worst' in terms of their vector-embeddings.

In [128]:
print_sorted_words('worst', metric='cosine')

Distance from 'worst':
0.000 - worst
0.059 - musicals
0.061 - compassion
0.065 - logical
0.072 - everywhere
0.076 - ha
0.078 - awful
0.079 - lacks
0.084 - chip
0.086 - meandering
...
1.243 - revealed
1.246 - 1940
1.255 - amazing
1.263 - funeral
1.270 - superb
1.273 - 7
1.298 - cameos
1.309 - eight
1.326 - fashion
1.394 - perfect


## Conclusion

This tutorial showed the basic methods for doing Natural Language Processing (NLP) using a Recurrent Neural Network with integer-tokens and an embedding layer. This was used to do sentiment analysis of movie reviews from IMDB. It works reasonably well if the hyper-parameters are chosen properly. But it is important to understand that this is not human-like comprehension of text. The system does not have any real understanding of the text. It is just a clever way of doing pattern-recognition.

## Exercises

These are a few suggestions for exercises that may help improve your skills with TensorFlow. It is important to get hands-on experience with TensorFlow in order to learn how to use it properly.

You may want to backup this Notebook before making any changes.

* Run more training-epochs. Does it improve performance?
* If your model overfits the training-data, try using dropout-layers and dropout inside the GRU.
* Increase or decrease the number of words in the vocabulary. This is done when the `Tokenizer` is initialized. Does it affect performance?
* Increase the size of the embedding-vectors to e.g. 200. Does it affect performance?
* Try varying all the different hyper-parameters for the Recurrent Neural Network.
* Use Bayesian Optimization from Tutorial #19 to find the best choice of hyper-parameters.
* Use 'post' for padding and truncating in `pad_sequences()`. Does it affect the performance?
* Use individual characters instead of tokenized words as the vocabulary. You can then use one-hot encoded vectors for each character instead of using the embedding-layer.
* Use `model.fit_generator()` instead of `model.fit()` and make your own data-generator, which creates a batch of data using a random subset of `x_train_tokens`. The sequences must be padded so they all match the length of the longest sequence.
* Explain to a friend how the program works.

## License (MIT)

Copyright (c) 2018 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.